### - 중간고사


본격적인 시작에 앞서, 필요한 외부 라이브러리 및 기타 명칭 정의



In [ ]:
import folium
import pandas as pd
import networkx as nx
import numpy as np


nodes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/수리적 빅데이터 개론/Daegu_node_1.csv")

nodes_x , nodes_y , nodes_id = nodes['X'], nodes['Y'] , nodes['NODE_ID']
# x는 경도 y는 위도

G = nx.Graph()
nodes1 = nodes[['NODE_ID', 'X', 'Y']]



y = nodes_y.mean()
x = nodes_x.mean()

daegu = folium.Map(location = [y,x] , zoom_start = 12) # 지도의 위도 / 경도 , 줌 상태
layer1 = folium.FeatureGroup(name = 'Reg')
layer2 = folium.FeatureGroup(name = 'DC')
layer3 = folium.FeatureGroup(name = 'CC')
layer4 = folium.FeatureGroup(name = 'BC')
layer5 = folium.FeatureGroup(name = 'Degree')






**데이터 전처리 과정(Daegu_nodes 내에 있는 nodes들 끼리의 Line만 표시하고 싶음)**











 F_link와 T_link는 node_id 를 이용하여 나타낸 점이다. 
이 때, node_id 내에 없는 T_linke와 F_link들을 삭제해주기 위해 
node_id내에 없는 F_link와 T_link들을 0으로(임의의 값) 바꿔준 후, 
0에 해당하는 index들을 뽑아내어서 그 index에 있는 F_link, T_link들을 삭제해준다.




In [ ]:
links = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/수리적 빅데이터 개론/Daegu_link_1.csv")

links_f , links_t , link_id = links['F_NODE'] , links['T_NODE'], links['LINK_ID']

nodes_id = list(nodes_id)


for i in range(len(links_f)):
  if links_f[i] not in nodes_id:
    links_f[i] = 0

for i in range(len(links_t)):
  if links_t[i] not in nodes_id:
    links_t[i] = 0

# node_id 에 없는(즉, 대구 밖에서 들어오거나 대구 밖으로 나가는) T,F link들을 임의의 값인 0으로 바꿔줌.


A = np.where(links_t == 0)  # T_link가 0인 index 추출
B = np.where(links_f == 0)  # F_link가 0인 index 추출
out_city = np.hstack([A, B]) # A, B 합치기
out_city = np.unique(out_city) # 중복 제거. 이제 이 index에 해당하는 부분들을 T_link, F_link에서 전부 지워주면 됨.

# np.delete 함수를 이용하기 위한 셋팅.
out_city = list(out_city)
links_f = np.array([links_f])
links_t = np.array([links_t])


links_f = np.delete(links_f, out_city)
links_t = np.delete(links_t, out_city)



print(len(links_f))
print(len(links_t))

# print(np.where(links_f == 0)) links_f, links_t 중 0을 값으로 가지는 인덱스가 있는지 확인
# print(np.where(links_t == 0))

links_f = list(links_f)
links_t = list(links_t) # 다시 list 형태로 복구



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


5665
5665





데이터 전처리가 끝났으니 이제 점을 찍고 연결해주어야함.



In [ ]:
# 그래프 그리기
for idx,row in nodes1.iterrows():
  # add node to Graph G
  G.add_node(row['NODE_ID'],latitude=row['Y'], longitude=row['X'])

for x in range(len(links_f)):
  G.add_edge(links_f[x], links_t[x])

DC = nx.degree_centrality(G)
CC = nx.closeness_centrality(G)
BC = nx.betweenness_centrality(G)

# 엣지 그리기

for i in range(len(links_f)):
  nodes_f = nodes_id.index(links_t[i])
  nodes_t = nodes_id.index(links_f[i])
  
  long1 = nodes_x[nodes_f]
  lati1 = nodes_y[nodes_f]
  
  long2 = nodes_x[nodes_t]
  lati2 = nodes_y[nodes_t]

  folium.PolyLine([(lati1, long1), (lati2, long2)], color = 'black', weight = 1, opacity = 0.5).add_to(layer1) 
  folium.PolyLine([(lati1, long1), (lati2, long2)], color = 'black', weight = 1, opacity = 0.5).add_to(layer2) 
  folium.PolyLine([(lati1, long1), (lati2, long2)], color = 'black', weight = 1, opacity = 0.5).add_to(layer3) 
  folium.PolyLine([(lati1, long1), (lati2, long2)], color = 'black', weight = 1, opacity = 0.5).add_to(layer4) 
  folium.PolyLine([(lati1, long1), (lati2, long2)], color = 'black', weight = 1, opacity = 0.5).add_to(layer5)
  

# 점찍기

for i in range(len(nodes)):
  folium.CircleMarker([nodes_y[i], nodes_x[i]], radius = 5, color = 'green',
                      popup = 'nodes_id : %d'%nodes_id[i]).add_to(layer1)
  
  folium.CircleMarker([nodes_y[i], nodes_x[i]], radius = DC[nodes_id[i]] * 3000, 
                      color = 'red', popup = 'DC of %d : %.6f'%(nodes_id[i], DC[nodes_id[i]])).add_to(layer2)

  folium.CircleMarker([nodes_y[i], nodes_x[i]], radius = CC[nodes_id[i]] * 100, 
                      color = 'blue', popup = 'CC of %d : %.6f'%(nodes_id[i], CC[nodes_id[i]])).add_to(layer3)

  folium.CircleMarker([nodes_y[i], nodes_x[i]], radius = BC[nodes_id[i]] * 100 , color = 'purple',
                      popup = 'BC of %d : %.6f'%(nodes_id[i], BC[nodes_id[i]])).add_to(layer4) 

  folium.CircleMarker([nodes_y[i], nodes_x[i]], radius = G.degree[nodes_id[i]] * 3 , color = 'yellow',
                      popup = 'degree of %d : %d'%(nodes_id[i], G.degree[nodes_id[i]])).add_to(layer5) 
                     




In [ ]:
daegu.add_child(layer1)

daegu.add_child(layer2)

daegu.add_child(layer3)

daegu.add_child(layer4)

daegu.add_child(layer5)

daegu.add_child(folium.LayerControl())



In [ ]:
daegu.save('dague.html')


In [ ]:

sum = 0
for i in DC.keys():
  sum += DC[i]
  
sum / len(DC)


0.001681551751860067

In [ ]:

sum = 0
for i in BC.keys():
  sum += BC[i]
 
sum / len(BC)


0.013020386495237743

In [ ]:

sum = 0
for i in CC.keys():
  sum += CC[i]
  
sum / len(CC)


0.040389928479988915